# Αποθήκευση και φόρτωση μοντέλων

Σε αυτό το notebook, θα δούμε πως μπορούμε να αποθηκεύσουμε και να φορτώσουμε δίκτυα μέσω της PyTorch. Οι λειτουργίες αυτές είναι πολύ σημαντικές καθώς πολύ συχνά θα φορτώνετε μοντέλα τα οποία έχετε εκπαιδεύσει πριν για να τα χρησιμοποιήσετε για προβλέψεις σε νέα δεδομένα ή για να συνεχίσετε την εκπαίδευσή τους σε επιπλέον δεδομένα.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import helper
import fc_model

In [ ]:
# Καθορίζω μετασχηματισμό για την κανονικοποίηση των δεδομένων
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# κατεβάζω και φορτώνω τα δεδομένα εκπαίδευσης
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# κατεβάζω και φορτώνω τα δεδομένα ελέγχου
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Εμφανίζω μία απο τις εικόνες.

In [ ]:
image, label = next(iter(trainloader))
helper.imshow(image[0,:]);

# Εκπαιδεύω το δίκτυο

Για να κάνω τα πράγματα πιο συνοπτικά εδώ, μετέφερα την αρχιτεκτονική του μοντέλου και τον κώδικα εκπαίδευσης από το τελευταίο notebook σε ένα αρχείο που ονομάζεται `fc_model`. Με την εισαγωγή αυτού, μπορούμε εύκολα να δημιουργήσουμε ένα πλήρως συνδεδεμένο δίκτυο με το `fc_model.Network` και να εκπαιδεύσουμε το δίκτυο χρησιμοποιώντας το `fc_model.train`. Θα χρησιμοποιήσω αυτό το μοντέλο (μόλις εκπαιδευτεί) για να δείξω πώς μπορούμε να αποθηκεύουμε και να φορτώνουμε μοντέλα.

In [ ]:
# Δημιουργήστε το δίκτυο, καθορίστε το κριτήριο και τον optimizer

model = fc_model.Network(784, 10, [512, 256, 128])
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
fc_model.train(model, trainloader, testloader, criterion, optimizer, epochs=2)

## Αποθήκευση και φόρτωση δικτύων

Όπως μπορείτε να φανταστείτε, δεν είναι καθόλου πρακτικό να εκπαιδεύετε κάθε φορά το δίκτυο σας όταν θέλετε να το χρησιμοποιήσετε. Αντ' αυτού, μπορούμε να αποθηκεύσουμε τα εκπαιδευμένα μας δίκτυα και μετά να τα φορτώσουμε αργότερα για να τα εκπαιδεύσουμε επιπλέον ή να τα χρησιμοποιήσουμε για προβλέψεις.

Οι παράμετροι για δίκτυα PyTorch αποθηκεύονται στο `state_dict` ενός μοντέλου. Μπορούμε να δούμε ότι το state dict περιέχει τους πίνακες βαρών και πόλωσης για κάθε ένα από τα επίπεδα μας.

In [ ]:
print("Το μοντέλο μας: \n\n", model, '\n')
print("Οι τιμές του state dict: \n\n", model.state_dict().keys())

Το απλούστερο πράγμα που πρέπει να κάνετε είναι απλά να σώσετε το state dict με `torch.save`. Για παράδειγμα, μπορούμε να το αποθηκεύσουμε σε ένα αρχείο `'checkpoint.pth'`.

In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')

Μετά μπορούμε να φορτώσουμε το state dict με `torch.load`.

In [ ]:
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

Και για να φορτώσετε το state dict στο δίκτυο, εκτελώ `model.load_state_dict(state_dict)`.

In [ ]:
model.load_state_dict(state_dict)

Φαίνεται αρκετά απλό, αλλά ως συνήθως είναι λίγο πιο περίπλοκο. Η φόρτωση του state dict λειτουργεί μόνο εάν η αρχιτεκτονική του μοντέλου είναι ακριβώς η ίδια με την αρχιτεκτονική του checkpoint. Εάν δημιουργήσω ένα μοντέλο με διαφορετική αρχιτεκτονική, αυτό αποτυγχάνει.

In [ ]:
# δοκιμαστε αυτη την εντολή
model = fc_model.Network(784, 10, [400, 200, 100])
# Αυτό θα προκαλέσει σφάλμα επειδή τα μεγέθη του τανυστή είναι λάθος!
model.load_state_dict(state_dict)

Αυτό σημαίνει ότι πρέπει να ξαναχτίσουμε το μοντέλο ακριβώς όπως ήταν όταν εκπαιδεύτηκε. Οι πληροφορίες σχετικά με την αρχιτεκτονική του μοντέλου πρέπει να αποθηκευτούν στο checkpoint, μαζί με το state dict. Για να το κάνετε αυτό, δημιουργείτε ένα λεξικό με όλες τις πληροφορίες που χρειάζεστε για να δημιουργήσετε εκ νέου το μοντέλο.

In [ ]:
checkpoint = {'input_size': 784,
              'output_size': 10,
              'hidden_layers': [each.out_features for each in model.hidden_layers],
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

Τώρα το checkpoint διαθέτει όλες τις απαραίτητες πληροφορίες για την ανοικοδόμηση του εκπαιδευμένου μοντέλου. Μπορείτε εύκολα να το υλοποιήσετε σε συνάρτηση αν θέλετε. Ομοίως, μπορούμε να γράψουμε μια συνάρτηση για τη φόρτωση των checkpoints.

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [ ]:
model = load_checkpoint('checkpoint.pth')
print(model)

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

Αυτό το notebook 📖 δημιουργήθηκε για το μάθημα ***Υπολογιστική Νοημοσύνη και Μηχανική Μάθηση*** του Τμήματος Μηχανικών Παραγωγής και Διοίκησης, της Πολυτεχνικής Σχολής του Δημοκριτείου Πανεπιστημίου Θράκης.<br>
This notebook is made available under the Creative Commons Attribution [(CC-BY)](https://creativecommons.org/licenses/by/4.0/legalcode) license. Code is also made available under the [MIT License](https://opensource.org/licenses/MIT).<br>
Author: Asst. Prof. Angelos Amanatiadis
<img src="assets/cc.png" style="width:55px; float: right; margin: 0px 0px 0px 0px;"></img>
<img src="assets/mit.png" style="width:40px; float: right; margin: 0px 10px 0px 0px;"></img>